In [60]:
# # !pip uninstall groupy -y
# !pip install GroupyAPI
# !pip install ratelimit
# !pip install wikipedia


In [68]:
#^ Dynamic updates through GroupMe

import ratelimit
from ratelimit import limits, sleep_and_retry
import wikipedia
import json
import time
import requests
import groupy
# from groupy import client as Client
from groupy.client import Client

@sleep_and_retry
def inner_calls(result):
    wikipedia_page = None
    try:
        print(f"Trying {result}...")
        query = result.lower()
        if ' All articles needing additional references' in query:
            pass
        elif ' All articles with unsourced statements' in query:
            pass
        elif 'import' == query[0:6]:
            pass
        elif 'list of' == query[0:8]:
            pass
        elif 'lists of' == query[0:9]:
            pass
        else:
            wikipedia_page = wikipedia.WikipediaPage(query) # get the Wikipedia page for the query. This will raise a PageError if the query is not found in Wikipedia.
            return wikipedia_page# break out of the loop if the query is found in Wikipedia
    except Exception:
        return wikipedia_page

#* bot functions for groupme
@ratelimit.sleep_and_retry
def get_closest_categories(query_from_groupme):
    """
    Get the closest categories for the given query.
    """
    # Try to find the closest Wikipedia page to the query.
    try:
        if isinstance(query_from_groupme, list):
            query_from_groupme = query_from_groupme[0]
        elif isinstance(query_from_groupme, str):
            pass
        else:
            raise Exception("The query must be a string or a list of strings.")
        if len(str(query_from_groupme)) >= 300:
            query_from_groupme = query_from_groupme[:300]
            print(f"Query was too long, so it was truncated to {query_from_groupme}.")
        search_results = wikipedia.search(query_from_groupme) # get the search results for the query from Wikipedia (this is a list of strings)
        #wikipedia_page = None # initialize the variable
        for result in search_results:
            wikipedia_page = inner_calls(result)
    except wikipedia.exceptions.PageError:
        # If the query is not found in Wikipedia, return a default category.
        # send a message to the thread saying this could not be found
        groupme_bot(f"Sorry, I couldn't find {query_from_groupme} in Wikipedia's categories.")
        return ["Miscellaneous"]
    except Exception as e:
        # If any other exception is raised, print the error message and return a default category.
        print(f"Other Error: {e}")
        return ["Miscellaneous"]
    
    try:
        # Get the categories for the Wikipedia page.
        wikipedia_categories = wikipedia_page.categories
        # Ignore any categories that contain the word "Category", "Wiki", or "wikipedia", as these are not useful categories.
        wikipedia_categories = [category for category in wikipedia_categories if 'category' not in category.lower() and 'wiki' not in category.lower() and 'wikipedia' not in category.lower()]
    except Exception as e:
        # If any exception is raised, print the error message and return a default category.
        print(f"Error: {e}")
        return ["Miscellaneous"]

    return wikipedia_categories



# ^ bot functions for groupme

@ratelimit.sleep_and_retry
def add_category(category_query):
    # test the category query to see if it is a valid Wikipedia category, and if so, add it to the categories file. Otherwise, ignore it.
    try:
        wikipedia.WikipediaPage(category_query).category
        is_valid_category = True
    except wikipedia.exceptions.PageError:
        is_valid_category = False
    except Exception as e:
        print(f"Error: {e}")
        is_valid_category = False
    
    # if the category query is a valid Wikipedia category, add it to the categories file
    if is_valid_category:
        with open("categories.txt", "a") as categories_file:
            categories_file.write(f"{category_query},\n")
        print(f"Added {category_query} to the categories file.")
        # send a message to the groupme thread acknowledging the addition of the category to the categories file
        groupme_bot(f"Added {category_query} to the categories file.")
    else:
        pass # do nothing if the category query is not a valid Wikipedia category

@ratelimit.sleep_and_retry
def setup_groupy():

    with open("./secrets.json") as json_file:
        secrets = json.load(json_file)
        bot_id = secrets["groupme_botid"]
        groupme_api = secrets["GROUPME_API_ACCESSTOKEN"]

    client = Client.from_token(groupme_api) # get the client object
    groups = list(client.groups.list_all()) # get the list of groups
    group = groups[0] # get the first group in the list of groups


    # open the categories file and get the list of categories
    with open("categories.txt", "r") as categories_file:
        categories = categories_file.readlines()
        categories = [category.strip() for category in categories]
    
    # get the messages in the group
    messages = list(group.messages.list_all())

    print(f"Let's see... who's playing today?")
    print(f"Here are the members of the group:")
    
    for member in group.members:
        print(f'\t{member.nickname}')
    
    # process the messages from group.members and populate the categories file
    # send a message to the group asking for categories
    #&groupme_bot("Please send me a list of Wikipedia categories that you would like me to use to generate questions. I will use these categories to generate questions for you. Please send me a list of categories, one per message. I will add each category to my list of categories. If you want to remove a category, send me a message with the category name preceded by a minus sign (-).")

    # get the messages from the group
    messages = list(group.messages.list_all())
    # filter the messages to those that were sent by users (i.e. not by the bot)
    user_messages = [message for message in messages if message.sender_id != bot_id]

    # now, we can process the new messages. Thank the specific user that posted the message, and add the category to the categories file
    for message in user_messages:
        # remove newlines from the message text, if any and nonAlphaNumeric characters
        message.text = message.text.replace("\n", "")
        message.text = message.text.replace(" ", "_")
        message.text = ''.join(e for e in message.text if e.isalnum() or e == '_') # remove nonAlphaNumeric characters, except for underscores, which are used in Wikipedia category names.
        # capitalize the first letter of the category name
        message.text = message.text[0].upper() + message.text[1:]
        
        # thank the user for the category
        if message.name != 'The Monikers Host' and message.name != 'GroupMe':
            groupme_bot(f"Thanks for the category, {message.name}!\n\t{message.text}")
            # add the category to the categories file
            add_category(message.text)
        else:
            pass

    print(f'I see that I sent {len(messages) - len(user_messages)} messages.')
    print(f'That means that users sent {len(user_messages)} messages.')
    # now, filter those messages to those we have not already processed (i.e. those that are not in the "messagesseen.txt" file).
    # first, get the list of messages we have already processed
    with open("messagesseen.txt", "r") as messages_seen_file:
        messages_seen = messages_seen_file.readlines()
        messages_seen = [message.strip() for message in messages_seen]
    
    # now, filter the user messages to those that are not in the messages seen file
    new_messages = [message for message in user_messages if message.id not in messages_seen]

    # now, add the new messages to the messages seen file
    with open("messagesseen.txt", "a") as messages_seen_file:
        for message in new_messages:
            messages_seen_file.write(f"{message.id}\n")
    
    # now, we can process the new messages
    # send a message acknowledging the new messages, and saying that they are being processed
    groupme_bot(f"Processing {len(new_messages)} updates to my knowledge base.")
    print(f"Processing {len(new_messages)} updates to my knowledge base.")

    new_messages = [message for message in new_messages if len(message.text) > 0 and message.text[0] != '-' and len(message.text) < 100] # filter out messages that are empty, start with a minus sign, or are too long (i.e. not categories

    
    print(f"working on {len(new_messages)} new messages")
    # process the new messages
    for message in new_messages:
        add_category(message.text) # add the category to the categories file, if it is a valid category
    

    # group = groups[0]
    # for member in group.members:
    #     print(member.nickname)

    for message in group.messages.list_all():
        print(message.text)
        for attachment in message.attachments:
            print(attachment.type)
        if 'love' in message.text.lower():
            message.like()

@ratelimit.sleep_and_retry
def groupme_bot(
    message_text="Welcome to the ever expanding world of Generative Monikers! I am your host, Hubert.",
):
    # Replace :bot_id with your bot's ID
    # read bot_id from secrets.json
    with open("./secrets.json") as json_file:
        secrets = json.load(json_file)
        bot_id = secrets["groupme_botid"]
        groupme_api = secrets["GROUPME_API_ACCESSTOKEN"]

    # if the message is not a single string, convert it to a string
    if type(message_text) != str:
        message_text = str(message_text)

    # Set the payload for the request
    payload = {"bot_id": bot_id, "text": message_text[0:1000]}

    # Make the POST request to the GroupMe API
    response = requests.post("https://api.groupme.com/v3/bots/post", json=payload)

    # Check the status code of the response
    if response.status_code != 202:
        print(f"Failed to send message: {response.status_code} {response.text}")
    else:
        # print("Message sent successfully.")
        pass


# function to check the groupme thread for new messages (that are not from the bot)
def check_for_new_messages():
    with open("./secrets.json") as json_file:
        secrets = json.load(json_file)
        bot_id = secrets["groupme_botid"]

    global groupme_api
    # access the GroupMe API and get the latest messages in the thread
    latest_messages = groupme_api.get_latest_messages()
    
    # filter the messages to get only those that are not from the bot
    new_messages = [message for message in latest_messages if message['sender_id'] != bot_id]
    
    return new_messages

@ratelimit.sleep_and_retry
def add_category(message):
    # check if the message is a valid Wikipedia category
    message = str(message)
    message = get_closest_categories(message)[0]
    try:
        wikipedia.WikipediaPage(message)
        is_valid_category = True
    except wikipedia.exceptions.PageError:
        print(f"PageError: {message} is not a valid Wikipedia category.")
        is_valid_category = False
    except wikipedia.exceptions.DisambiguationError:
        print(f"DisambiguationError: {message} is not a valid Wikipedia category.")
        is_valid_category = False
    except Exception as e:
        print(f"Unknown error: {e}")
        is_valid_category = False

    # if the message is a valid Wikipedia category, add it to the categories text file
    if is_valid_category:
        with open("categories.txt", "a") as categories_file:
            categories_file.write(f"{message}\n")
    # if the message is not a valid Wikipedia category, ignore it
    else:
        pass



In [69]:
setup_groupy()

Let's see... who's playing today?
Here are the members of the group:
	Graham Waters
Trying History...
Trying HIStory: Past, Present and Future, Book I...
Trying History (disambiguation)...


/opt/anaconda3/envs/groupme/lib/python3.9/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/anaconda3/envs/groupme/lib/python3.9/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Trying World history...
Trying History of Europe...
Trying Ancient history...
Trying Jewish history...
Trying History of Russia...
Trying History of India...
Trying Alternate history...
PageError: All articles that may contain original research is not a valid Wikipedia category.
Query was too long, so it was truncated to Import_requests_Set_the_base_URL_for_the_Wikipedia_APIapi_url__httpsenwikipediaorgwapiphp_Set_the_search_query_parametersparams______action_query____format_json____list_search____utf8_1____formatversion_2____srsearch_legal_maxim____format_json_Make_the_HTTP_GET_request_to_the_Wikipedia_APIresponse__.
Error: local variable 'wikipedia_page' referenced before assignment
Error: local variable 'wikipedia_page' referenced before assignment
Unknown error: HTTPConnectionPool(host='en.wikipedia.org', port=80): Read timed out. (read timeout=None)
Query was too long, so it was truncated to From_nltkmetricsdistance_import_edit_distancedef_find_similar_wordstitle_definition_thresho

KeyboardInterrupt: 

---

----

In [ ]:
add_category("Pixar_animated_films") #note: this is already formatted right for the Wikipedia API. spaces are replaced with underscores, and the first letter is capitalized. When refactoring the add_category function, make sure to add this functionality to the function for nonconformist category queries.
